# Develops the Calculation Model for the Online Calculator

In [95]:
import sys
from math import nan, isnan, inf
import numbers
from importlib import reload
import inspect
from pprint import pprint

import pandas as pd
import numpy as np
# import matplotlib pyplot commands
from matplotlib.pyplot import *
from IPython.display import Image, Markdown
from qgrid import show_grid as sh
 
# Show Plots in the Notebook
%matplotlib inline
 
# 'style' the plot like fivethirtyeight.com website
style.use('bmh')

In [2]:
rcParams['figure.figsize']= (10, 6)   # set Chart Size
rcParams['font.size'] = 14            # set Font size in Chart

In [3]:
# Access the directory where some utility modules are located in the
# actual heat pump calculator.
sys.path.insert(0, '../../heat-pump-calc/heatpump/')

In [4]:
import library as lib
reload(lib)

<module 'library' from '../../heat-pump-calc/heatpump/library.py'>

In [28]:
sh(lib.df_city[['Name']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [80]:
lib.fuels()

[('Electricity', 1),
 ('Natural Gas', 2),
 ('Propane', 3),
 ('#1 Oil', 4),
 ('#2 Oil', 5),
 ('Birch Wood', 6),
 ('Spruce Wood', 7),
 ('Wood Pellets', 8),
 ('Coal', 9),
 ('District Heat, Steam', 10),
 ('District Heat, Hot Water', 11)]

In [79]:
lib.city_from_id(45)

Name                                                            Ambler
Latitude                                                        67.087
Longitude                                                     -157.859
ERHRegionID                                                          5
WAPRegionID                                                          5
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                          NaN
Oil2Price                                                          NaN
PropanePrice                                                       NaN
BirchPrice                                                         NaN
SprucePrice                                                        NaN
CoalPrice                                                          NaN
SteamPrice                                                         NaN
HotWat

In [127]:
lib.heat_pump_from_id(575)

brand                 Fujitsu
ahri_num              8703350
zones                  Single
outdoor_model       AOU12RLS3
indoor_model       ASU12RLS3Y
hspf                       14
in_pwr_5F_max             2.1
capacity_5F_max         16500
in_pwr_47F_min            0.2
Name: 575, dtype: object

## Object that Calculates Monthly Electric Utility Costs 

In [6]:
def chg_nonnum(val, sub_val):
    """Changes a nan or anything that is not a number to 'sub_val'.  
    Otherwise returns val.
    """
    if isinstance(val, numbers.Number):
        if isnan(val):
            return sub_val
        else:
            return val
    else:
        return sub_val


class ElecCostCalc:
    """Class to calculate monthly electric cost given a block rate structure possibly including
    PCE.
    """

    def __init__(
        self,
        utility, 
        sales_tax=0.0,      
        pce_limit=500.0,
        ):
        """Constructor parameters:
        utility:            The electric utility object from the library module.
        sales_tax:          sales tax to apply to total electric bill.
        pce_limit:          maximum number kWh that PCE can apply to. 
                                use math.nan or math.inf to indicate no limit.
        """
        # save some of the variables for use in other methods
        # add Sales Tax to any rate elements
        self.utility = utility
        self.sales_tax = sales_tax
        
        # Make a set of blocks that:
        #   * have the quantity of kWh in the block instead of the block upper limit kWh
        #   * include a block to accomodate a PCE limit
        #   * account for the effects of PCE

        # Convert PCE limit to infinite if no limit
        pce_limit = chg_nonnum(pce_limit, inf)

        if pce_limit > 0:
            # get the correct PCE rate and convert anything not a number to 0.
            pce_adj = chg_nonnum(self.utility.PCE, 0.0)
        else:
            pce_adj = 0.0

        # Flag to indicate whether a PCE block has been added.  If PCE is zero
        # no need to add a block.        
        pce_block_added = (pce_adj==0.0)  

        # Make a new set of blocks that includes the PCE limit as a new block
        # and the set is only as long as it needs to be.
        selected_blocks = self.utility.Blocks
        done = False
        temp_blocks = []
        for max_kwh, rate in selected_blocks:
            b_kwh = chg_nonnum(max_kwh, inf)
            done = (b_kwh == inf)

            # Determine whether it is time to insert the PCE block or not.
            # Test is whether block quantity exceeds the PCE limit.
            if not pce_block_added and (b_kwh > pce_limit):
                # Need to insert the PCE block now.
                temp_blocks.append( (pce_limit, rate) )
                pce_block_added = True          # PCE block has now been added.

            # address the case where the PCE limit matches the upper block
            if pce_limit == b_kwh:
                pce_block_added = True

            # add the block
            temp_blocks.append( (b_kwh, rate))

            if done:
                break

        # Include the PCE adjustment, and sales tax.
        # Also, convert the block quantities so that they are the quantity of 
        # kWh in the block instead of the upper limit of the block.
        prev_upper = 0.0
        self._blocks = []    # final block list
        for max_kwh, rate in temp_blocks:
            if max_kwh <= pce_limit:
                rate -= pce_adj
            rate *= (1. + sales_tax)
            self._blocks.append( (max_kwh - prev_upper, rate) )
            prev_upper = max_kwh

    def monthly_cost(self, kwh_energy, kw_demand=0.0):
        """Returns the total electric cost for a month, given energy usage of
        'kwh_energy' and peak demand of 'kw_demand'.
        """
        # customer charge
        cost = chg_nonnum(self.utility.CustomerChg, 0.0) * (1. + self.sales_tax)

        # demand charge
        cost += kw_demand * chg_nonnum(self.utility.DemandCharge, 0.0) * (1. + self.sales_tax)

        remaining_kwh = kwh_energy
        for b_kwh, b_rate in self._blocks:
            kwh_in_block = min(remaining_kwh, b_kwh)
            cost += kwh_in_block * b_rate
            remaining_kwh -= kwh_in_block
            if remaining_kwh < 0.1:         # 0.1 kWh in case of rounding issues
                break
        
        return cost

    def final_blocks(self):
        """Debug method to return underlying rate blocks
        """
        return self._blocks

In [7]:
util_id = 383
utility = lib.util_from_id(util_id)
ec = ElecCostCalc(utility, sales_tax=0.0, pce_limit=500.0)
print(ec.utility.Blocks)
ec.final_blocks()

[(100.0, 0.817300021299161), (400.0, 0.742400020011701), (500.0, 0.667500018724241), (1000.0, 0.592500000842847), (nan, 0.562600016244688)]


[(100.0, 0.44630002940539304),
 (300.0, 0.37140002811793305),
 (100.0, 0.29650002683047305),
 (500.0, 0.592500000842847),
 (inf, 0.562600016244688)]

In [8]:
ec.monthly_cost(401)

156.34651140274968

In [9]:
util_id = 1
utility = lib.util_from_id(util_id)
ec = ElecCostCalc(utility, sales_tax=0.0, pce_limit=500.0)
print(ec.final_blocks())
utility

[(inf, 0.18647000112105194)]


ID                                                              1
Name                                Chugach Electric- Residential
Active                                                       True
Type                                                            1
IsCommercial                                                False
ChargesRCC                                                   True
PCE                                                             0
CO2                                                           1.1
CustomerChg                                                     8
DemandCharge                                                  NaN
NameShort                                                  Chugac
Blocks          [(nan, 0.18647000112105194), (nan, nan), (nan,...
Name: 1, dtype: object

In [10]:
ec.monthly_cost(0, 1)

8.0

In [11]:
custom_ut = utility.copy()
custom_ut['DemandCharge'] = 10.0
ec = ElecCostCalc(custom_ut, sales_tax=0.0, pce_limit=500.0)
ec.monthly_cost(0, 1.0)

18.0

## Home Space Heating Simulation Model

In [146]:
class HomeEnergyModel(object):
    
    def __init__(self,
                 city_id,
                 hp_model_id,
                 exist_heat_fuel_id,
                 exist_heat_effic,
                 exist_kwh_per_mmbtu,     # Boiler: 5.5, Toyo: 3, Oil Furnace: 8.75 - 15
                 exist_is_point_source,
                 co2_lbs_per_kwh,
                 low_temp_cutoff,
                 garage_stall_count,
                 garage_heated_by_hp,
                 bldg_floor_area,
                 indoor_heat_setpoint,
                 insul_level,             # 1 - 2x4, 2 - 2x6, 3 - better than 2x6 Walls
                 pct_exposed_to_hp,
                 doors_open_to_adjacent,
                 low_temp_acceptance,     # 1 - no temp drop in back rooms, 2 - 4 deg F cooler, 10 deg F cooler

                 # The inputs below are not user inputs, they control the 
                 # calculation process. They are give default values.
                 hp_only=False,           # when using heat pump, it's the only heat source for home
                 no_heat_pump_use=False,  # If True, models existing heating system alone.
                 ua_true_up=1.0,          # used to true up calculaion to actual fuel use
                 **kwargs
                ):
        
        # Store all of these input parameters as object attributes
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        for arg in args[1:]:
            setattr(self, arg, values[arg])

    def __repr__(self):
        """Returns a string with all the object attributes shown.  Text is truncated
        at 1,000 characters for attributes with long representations.
        """
        s = ''
        for attr in self.__dict__:
            val = repr(self.__dict__[attr])[:1000]
            if len(val)>70:
                s+=f'\n{attr}:\n{val}\n\n'
            else:
                s += f'{attr}: {val}\n'
        return s
    
    # ------- Data needed for calculation of COP vs. Temperature
    
    # Piecewise linear COP vs. outdoor temperature.
    COP_vs_TEMP = (
        (-20.0, 1.1),
        (0.0, 2.0),
        (10.0, 2.2),
        (15.0, 2.3),
        (20.0, 2.5),
        (25.0, 2.7),
        (30.0, 2.8),
        (40.0, 3.0),
        (50.0, 3.2)
    )

    # convert to separate lists of temperatures and COPs
    TEMPS_FIT, COPS_FIT = tuple(zip(*COP_vs_TEMP))

    # The HSPF value that this curve is associated with
    BASE_HSPF = 13.3
    
    # -------------- OTHER CONSTANTS ---------------
    
    GARAGE_HEATING_SETPT = 55.0    # deg F
    
    # ---------------- Main Calculation Method --------------------
    
    def calculate(self):
        """Main calculation routine that models the home and determines
        loads and fuel use by hour.  Also calculates summary results.
        """
        # Set up some shortcut variables to shorten calculation code
        s = self
        
        # ------ Create object variables from the input IDs
        s.city = lib.city_from_id(s.city_id)
        s.hp_model = lib.heat_pump_from_id(s.hp_model_id)
        s.exist_heat_fuel = lib.fuel_from_id(s.exist_heat_fuel_id)
        
        # ------ Make a DataFrame with hourly input information
        # Do as much processing at this level using array operations, as
        # opposed to processing within the hourly loop further below.
        
        df_tmy = lib.tmy_from_id(self.city.TMYid)
        s.df_hourly = df_tmy[['db_temp']].copy()
        # and now make a shorthand variable for this DataFrame
        dfh = s.df_hourly
        dfh['day_of_year'] = dfh.index.dayofyear
        dfh['month'] = dfh.index.month

        # Determine days that the heat pump is running.  Look at the minimum
        # temperature for the day, and ensure that it is above the low 
        # temperature cutoff.
        hp_is_running = lambda x: (x.min() > self.low_temp_cutoff)
        dfh['running'] = dfh.groupby('day_of_year')['db_temp'].transform(hp_is_running)

        # Determine a heat pump COP for each hour
        # ** NEED TO ADJUST FOR INDOOR SETPOINT and MOUNTING LOCATION OF INDOOR UNITS 
        cop_interp = np.interp(dfh.db_temp, 
                               HomeEnergyModel.TEMPS_FIT, 
                               HomeEnergyModel.COPS_FIT)
        dfh['cop'] = cop_interp * self.hp_model.hspf / HomeEnergyModel.BASE_HSPF

        # adjustment to UA for insulation level.  My estimate, accounting
        # for better insulation *and* air-tightness as you move up the 
        # insulation scale.
        ua_insul_adj_arr = np.array([1.25, 1.0, 0.75])   # the adjustment factors by insulation level
        ua_insul_adj = ua_insul_adj_arr[s.insul_level - 1]   # pick the appropriate one
        
        # The UA values below are Btu/hr/deg-F
        # This is the UA / ft2 of the Level 2 (ua_insul_adj = 1) home
        # for the main living space.  Assume garage UA is about 10% higher
        # due to higher air leakage.
        # Determined this UA/ft2 below by modeling a typical Enstar home
        # and having the model estimate space heating use of about 1250 CCF.
        # See 'accessible_UA.ipynb'.
        ua_per_ft2 = 0.189
        ua_home = ua_per_ft2 * ua_insul_adj * s.bldg_floor_area * s.ua_true_up
        garage_area = (0, 14*22, 22*22, 36*25, 48*28)[s.garage_stall_count]
        ua_garage = ua_per_ft2 * 1.1 * ua_insul_adj * garage_area * s.ua_true_up
        
        # Save these UA values as object attributes
        s.ua_home = ua_home
        s.ua_garage = ua_garage

        # Balance Points of main home and garage
        # Assume a 10 deg F internal/solar heating effect for Level 2 insulation
        # in the main home and a 5 deg F heating effect in the garage.
        # Adjust the heating effect accordingly for other levels of insulation.
        htg_effect = np.array([10., 10., 10.]) / ua_insul_adj_arr
        balance_point_home = s.indoor_heat_setpoint - htg_effect[s.insul_level - 1]
        htg_effect = np.array([5.0, 5.0, 5.0]) / ua_insul_adj_arr  # fewer internal/solar in garage
        balance_point_garage = HomeEnergyModel.GARAGE_HEATING_SETPT - htg_effect[s.insul_level - 1]

        # BTU loads in the hour for the heat pump and for the secondary system.
        hp_load = []
        secondary_load = []

        # More complicated calculations are done in this hourly loop.  If processing
        # time becomes a problem, try to convert the calculations below into array
        # operations that can be done outside the loop.
        s.max_hp_reached = False       # tracks whether heat pump max output has been reached.
        for h in dfh.itertuples():
            # calculate total heat load for the hour.
            # Really need to recognize that delta-T to outdoors is lower in the adjacent and remote spaces
            # if there heat pump is the only source of heat.
            home_load = max(0.0, balance_point_home - h.db_temp) * ua_home 
            garage_load = max(0.0, balance_point_garage - h.db_temp) * ua_garage
            total_load = home_load + garage_load
            if not h.running or s.no_heat_pump_use:
                hp_load.append(0.0)
                secondary_load.append(total_load)
            else:
                max_hp_output = s.hp_model.in_pwr_5F_max * h.cop * 3412.
                if s.hp_only:
                    hp_ld = min(home_load + garage_load * s.garage_heated_by_hp, max_hp_output)                    
                    hp_load.append(hp_ld)
                    secondary_load.append(total_load - hp_load)
                else:
                    # Nowhere near correct yet.  Just get a calc framework working.
                    hp_ld = min(home_load * s.pct_exposed_to_hp + garage_load * s.garage_heated_by_hp, max_hp_output)
                    hp_load.append(hp_ld)
                    secondary_load.append(total_load - hp_ld)
                if hp_ld >= max_hp_output * 0.999:
                    # running at within 0.1% of maximum heat pump output.
                    s.max_hp_reached = True

        dfh['hp_load_mmbtu'] = np.array(hp_load) / 1e6
        dfh['secondary_load_mmbtu'] = np.array(secondary_load) / 1e6

        # using array operations, calculate kWh use by the heat pump and 
        # the Btu use of secondary system.
        dfh['hp_kwh'] = dfh.hp_load_mmbtu / dfh.cop / 0.003412
        dfh['secondary_fuel_mmbtu'] = dfh.secondary_load_mmbtu / s.exist_heat_effic
        dfh['secondary_kwh'] = dfh.secondary_load_mmbtu * s.exist_kwh_per_mmbtu  # auxiliary electric use
        
        # Store annual and monthly totals.
        # Annual totals is a Pandas Series.
        total_cols = ['hp_load_mmbtu', 'secondary_load_mmbtu', 'hp_kwh', 'secondary_fuel_mmbtu', 'secondary_kwh']
        s.df_monthly = dfh.groupby('month')[total_cols].sum()
        dfm = s.df_monthly    # shortcut variable
        
        # Add in a column for the peak heat pump demand during the month
        dfm['hp_kw'] = dfh.groupby('month')[['hp_kwh']].max()
        
        # physical units for secondary fuel
        fuel = self.exist_heat_fuel
        dfm['secondary_fuel_units'] = dfm['secondary_fuel_mmbtu'] / fuel.btus * 1e6

        # COP by month
        dfm['cop'] = dfm.hp_load_mmbtu / (dfm.hp_kwh * 0.003412)   

        # Total kWh, heat pump + auxiliary of secondary system
        dfm['total_kwh'] = dfm.hp_kwh + dfm.secondary_kwh
                    
        # Total lbs of CO2 per month, counting electricity and fuel
        dfm['co2_lbs'] = dfm.total_kwh * s.co2_lbs_per_kwh + dfm.secondary_fuel_mmbtu * fuel.co2

        # Change index to Month labels
        s.df_monthly.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        s.ser_annual = s.df_monthly.sum().drop(['cop'])
        # Add the seasonal COP to the annual results
        tot = s.ser_annual
        if tot.hp_kwh:
            s.ser_annual['cop'] =  tot.hp_load_mmbtu * 1e6 / tot.hp_kwh / 3412.
        else:
            s.ser_annual['cop'] = np.nan
        
    def monthly_results(self):
        """Returns a DataFrame of monthly results.
        """
        return self.df_monthly
    
    def annual_results(self):
        return self.ser_annual
                    
    def design_heat_load(self):
        """Returns the 99% design heat load for the building and the associated
        design temperature, including the garage if present.  Do not account for 
        any internal or solar gains, as is conventional.
        Return values are Btu/hour and deg F. 
        """
        # get the 1% outdoor temperature
        design_temp = self.df_hourly.db_temp.quantile(0.01)
        design_load = self.ua_home * (self.indoor_heat_setpoint - design_temp) + \
                      self.ua_garage * (HomeEnergyModel.GARAGE_HEATING_SETPT - design_temp)
        return design_load, design_temp
    
    def hp_max_capacity_5F(self):
        """Returns the maximum capacity of the heat pump at 5 deg F.  Uses the
        COP curve unless the manufacturer's spec is lower.
        Returns is Btu/hour.
        """
        cop_5F = np.interp(5.0, HomeEnergyModel.TEMPS_FIT, HomeEnergyModel.COPS_FIT)
        max_from_curve = cop_5F * self.hp_model.in_pwr_5F_max * 3412.
        return min(max_from_curve, self.hp_model.capacity_5F_max)
        

## Main Controller Model that Runs Necessary Simulations to Estimate Impact of Heat Pump

In [179]:
"""Provides a class to model the impact of a heat pump on
energy use and cost.
"""

def make_pattern(esc, life):
    """Makes a numpy array of length (life + 1) containing an escalation pattern
    that starts with a 1.0 in year 1 and escalates at the rate of 'esc' per year.
    """
    pat = np.ones(life - 1) * (1 + esc)
    return np.insert(pat.cumprod(), 0, [0.0, 1.0])

def convert_co2_to_miles_driven(co2_saved):
    """Converts CO2 emissions to a mileage driven
    equivalent for vehicles in the U.S. using EPA
    methodology:  https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references#miles
    """
    pounds_in_metric_ton = 2204.62
    tons_co2_per_gallon = 0.0089
    avg_gas_mileage_us_fleet = 22
    mileage_equivalent = co2_saved / pounds_in_metric_ton / tons_co2_per_gallon * avg_gas_mileage_us_fleet
    
    return mileage_equivalent

class HP_model:

    def __init__(self,
                 city_id,
                 utility_id,
                 co2_lbs_per_kwh,
                 exist_heat_fuel_id,
                 exist_unit_fuel_cost,
                 exist_fuel_use,
                 exist_heat_effic,
                 exist_kwh_per_mmbtu,
                 exist_is_point_source,
                 includes_dhw,
                 dhw_occupant_count,
                 includes_dryer,
                 elec_use_jan,
                 elec_use_may,
                 hp_model_id,
                 low_temp_cutoff,
                 garage_stall_count,
                 garage_heated_by_hp,
                 bldg_floor_area,
                 indoor_heat_setpoint,
                 insul_level,  
                 pct_exposed_to_hp,
                 doors_open_to_adjacent,
                 low_temp_acceptance,
                 capital_cost,
                 rebate_dol,
                 pct_financed,
                 loan_term,
                 loan_interest,
                 hp_life,
                 op_cost_chg,
                 sales_tax,
                 discount_rate,
                 inflation_rate,
                 fuel_esc_rate,
                 elec_esc_rate,
                ):

        # Store all of these input parameters as object attributes.
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        for arg in args[1:]:
            setattr(self, arg, values[arg])
            
        # Look up the objects associated with the IDs
        self.city = lib.city_from_id(city_id)
        self.utility = lib.util_from_id(utility_id)
        self.exist_fuel = lib.fuel_from_id(exist_heat_fuel_id)
        self.hp_model = lib.heat_pump_from_id(hp_model_id)
                    
    def __repr__(self):
        """Returns a string with all the object attributes shown.  Text is truncated
        at 1,000 characters for attributes with long representations.
        """
        s = ''
        for attr in self.__dict__:
            val = repr(self.__dict__[attr])[:1000]
            if len(val)>70:
                s+=f'\n{attr}:\n{val}\n\n'
            else:
                s += f'{attr}: {val}\n'
        return s
        
    def check_inputs(self):
        pass
        
    def run(self):
        
        # shortcut for self
        s = self
        
        # shortcut to existing heating fuel
        fuel = s.exist_fuel

        # holds summary measures for the heat pump project (e.g. seasonal COP,
        # internal rate of return).  Fill out first item: secondary fuel info.
        s.summary = {'fuel': fuel}
        
        # Create the home energy simulation object
        sim = HomeEnergyModel(
            city_id=s.city_id,
            hp_model_id=s.hp_model_id,
            exist_heat_fuel_id=s.exist_heat_fuel_id,
            exist_heat_effic=s.exist_heat_effic,
            exist_kwh_per_mmbtu=s.exist_kwh_per_mmbtu,    
            exist_is_point_source=s.exist_is_point_source,
            co2_lbs_per_kwh=s.co2_lbs_per_kwh,
            low_temp_cutoff=s.low_temp_cutoff,
            garage_stall_count=s.garage_stall_count,
            garage_heated_by_hp=s.garage_heated_by_hp,
            bldg_floor_area=s.bldg_floor_area,
            indoor_heat_setpoint=s.indoor_heat_setpoint,
            insul_level=s.insul_level,            
            pct_exposed_to_hp=s.pct_exposed_to_hp,
            doors_open_to_adjacent=s.doors_open_to_adjacent,
            low_temp_acceptance=s.low_temp_acceptance,    
        )
        
        # Match the existing space heating use if it is provided.  Do so by using
        # the UA true up factor.
        # **** TO DO Deal with Electric Heat.  Calculate a space_fuel_use for electric
        # **** and then use the same calculation below.
        if s.exist_fuel_use is not None:
            
            # Remove DHW and Clothes dryer if they are present in the fuel use
            # number.
            space_fuel_use = s.exist_fuel_use
            if s.includes_dhw:
                dhw_use = s.dhw_occupant_count * 4.23e6 / fuel.dhw_effic / fuel.btus
                space_fuel_use -= dhw_use
                
            if s.includes_dryer:
                space_fuel_use -= 2.15e6 * s.dhw_occupant_count / fuel.btus
            
            sim.no_heat_pump_use = True
            sim.calculate()
            fuel_use1 = sim.annual_results().secondary_fuel_units
            
            # scale the UA linearly to attempt to match the target fuel use
            ua_true_up = space_fuel_use / fuel_use1
            sim.ua_true_up = ua_true_up
            sim.calculate()
            fuel_use2 = sim.annual_results().secondary_fuel_units
            
            # In case it wasn't linear, inter/extrapolate to the final ua_true_up
            slope = (fuel_use2 - fuel_use1)/(ua_true_up - 1.0)
            ua_true_up = 1.0 + (space_fuel_use - fuel_use1) / slope

        else:
            ua_true_up = 1.0
            
        # Set the UA true up value into the model and also save it as
        # an attribute of this object so it can be observed.
        sim.ua_true_up = ua_true_up
        s.ua_true_up = ua_true_up
        
        # Run the base case with no heat pump and record energy results
        sim.no_heat_pump_use = True
        sim.calculate()
        s.df_mo_en_base = sim.monthly_results()
        
        # Run the model with the heat pump and record energy results
        sim.no_heat_pump_use = False
        sim.calculate()
        s.df_mo_en_hp = sim.monthly_results()
        # record design heat load
        s.summary['design_heat_load'], s.summary['design_heat_temp'] = sim.design_heat_load()
        
        # Calculate some summary measures
        s.summary['cop'] = sim.annual_results().cop
        s.summary['hp_max_capacity_5F'] = sim.hp_max_capacity_5F()
        s.summary['max_hp_reached'] = sim.max_hp_reached
        # CO2 savings
        ann_en_base = s.df_mo_en_base.sum()
        ann_en_hp = s.df_mo_en_hp.sum()
        s.summary['co2_lbs_saved'] = ann_en_base.co2_lbs - ann_en_hp.co2_lbs
        s.summary['co2_driving_miles_saved'] = convert_co2_to_miles_driven(s.summary['co2_lbs_saved'])
        
        # Create DataFrames that hold monthly energy cost amounts
        self.calc_monthly_cash()
        
        # Create a multi-year Cash Flow DataFrame and summary economic measures.
        self.calc_cash_flow()
            
    def calc_monthly_cash(self):
        """Calculates two DataFrames, s.df_mo_dol_base and s.df_mo_dol_hp, that contain
        the fuel and electricity costs in the base case (no heat pump) scenario and the
        with heat pump scenario.  A number of inputs found as object attributes are used. 
        """
        # shortcut to self
        s = self

        # The User supplied a January and a May kWh usage value that should
        # be used for the base case (no heat pump) total electricity use.
        # But, need to come up with a kWh value for every month.  Do that by
        # adjusting the kWh pattern available for this city.
        #
        # Determine the multiplier to adjust to the pattern to the actual.
        pat_use = np.array(s.city.avg_elec_usage)
        mult = (s.elec_use_jan - s.elec_use_may) / (pat_use[0] - pat_use[4])
        pat_use = mult * pat_use
        pat_use += s.elec_use_jan - pat_use[0]
        
        # Start the DataFrames, base and w/ heat pump
        # Each starts with just an index column with the month
        # Make shortcut variables as well.
        s.df_mo_dol_base = dfb = s.df_mo_en_base[[]].copy()
        s.df_mo_dol_hp = dfh = s.df_mo_en_base[[]].copy()
        
        # Make an object to calculate electric utility costs
        elec_cost_calc = ElecCostCalc(s.utility, sales_tax=s.sales_tax, pce_limit=500.0)
        # cost function that will be applied to each row of the cost DataFrame
        cost_func = lambda r: elec_cost_calc.monthly_cost(r.elec_kwh, r.elec_kw)

        # Now the no-PCE version
        util_no_pce = s.utility.copy()
        util_no_pce.PCE = 0.0    # blank out the PCE
        elec_cost_calc_no_pce = ElecCostCalc(util_no_pce, sales_tax=s.sales_tax, pce_limit=500.0)
        cost_func_no_pce = lambda r: elec_cost_calc_no_pce.monthly_cost(r.elec_kwh, r.elec_kw)

        dfb['elec_kwh'] =  pat_use
        # rough estimate of a base demand: not super critical, as the demand rate 
        # structure does not have blocks.  Assume a load factor of 0.4
        dfb['elec_kw'] = dfb.elec_kwh / 730.0 / 0.4
        dfb['elec_dol'] = dfb.apply(cost_func, axis=1)
        dfb['elec_dol_no_pce'] = dfb.apply(cost_func_no_pce, axis=1)

        # Now fuel
        dfb['secondary_fuel_units'] = s.df_mo_en_base.secondary_fuel_units
        dfb['secondary_fuel_dol'] = dfb.secondary_fuel_units * s.exist_unit_fuel_cost * (1. + s.sales_tax)

        # Total Electric + space heat
        dfb['total_dol'] =  dfb.elec_dol + dfb.secondary_fuel_dol
        dfb['total_dol_no_pce'] =  dfb.elec_dol_no_pce + dfb.secondary_fuel_dol

        # Now with the heat pump
        # determine extra kWh used in the heat pump scenario
        extra_kwh = (s.df_mo_en_hp.total_kwh - s.df_mo_en_base.total_kwh).values
        dfh['elec_kwh'] = dfb['elec_kwh'] + extra_kwh
        dfh['elec_kw'] =  dfb['elec_kw'] + s.df_mo_en_hp.hp_kw
        dfh['elec_dol'] = dfh.apply(cost_func, axis=1)
        dfh['elec_dol_no_pce'] = dfh.apply(cost_func_no_pce, axis=1)

        # Now fuel
        dfh['secondary_fuel_units'] = s.df_mo_en_hp.secondary_fuel_units
        dfh['secondary_fuel_dol'] = dfh.secondary_fuel_units * s.exist_unit_fuel_cost * (1. + s.sales_tax)

        # Total Electric + space heat
        dfh['total_dol'] =  dfh.elec_dol + dfh.secondary_fuel_dol
        dfh['total_dol_no_pce'] =  dfh.elec_dol_no_pce + dfh.secondary_fuel_dol
        
    def calc_cash_flow(self):
        """Calculates the cash flow impacts of the installation over the 
        life of the heat pump.  Creates a DataFrame, self.df_cash_flow, that shows
        the impacts. In that DataFrame, postive values are benefits and negative 
        values are costs. 
        Also calculates some summary economic measures that are added to
        the self.summary dictionary.
        """
        s = self   # shortcut variable

        # determine the changes caused by the heat pump on an annual basis.
        # First calculate annual totals for base case and heat pump case and
        # then calculate the change.
        ann_base = s.df_mo_dol_base.sum()
        ann_hp = s.df_mo_dol_hp.sum()
        ann_chg = ann_hp - ann_base
        initial_cost = np.zeros(s.hp_life+1)
        
        initial_cost[0] = -s.capital_cost * (1 + s.sales_tax) * (1 - s.pct_financed) + s.rebate_dol
        loan_pmt = np.pmt(s.loan_interest, s.loan_term, s.capital_cost * (1 + s.sales_tax) * s.pct_financed)
        loan_cost = [0.0] + [loan_pmt] * s.loan_term + [0.0] * (s.hp_life -  s.loan_term)
        loan_cost = np.array(loan_cost)
        operating_cost = -s.op_cost_chg * make_pattern(s.inflation_rate, s.hp_life)
        fuel_cost = -ann_chg.secondary_fuel_dol * make_pattern(s.fuel_esc_rate, s.hp_life)
        elec_cost = -ann_chg.elec_dol * make_pattern(s.elec_esc_rate, s.hp_life)
        cash_flow = initial_cost + loan_cost + operating_cost + fuel_cost + elec_cost
        
        # Do the two columns that change when PCE is ignored
        elec_cost_no_pce = -ann_chg.elec_dol_no_pce * make_pattern(s.elec_esc_rate, s.hp_life)
        cash_flow_no_pce = initial_cost + loan_cost + operating_cost + fuel_cost + elec_cost_no_pce
        
        s.df_cash_flow = pd.DataFrame(
            {'initial_cost': initial_cost,
             'loan_cost': loan_cost,
             'operating_cost': operating_cost,
             'fuel_cost': fuel_cost,
             'elec_cost': elec_cost,
             'elec_cost_no_pce': elec_cost_no_pce,
             'cash_flow': cash_flow,
             'cash_flow_no_pce': cash_flow_no_pce
            }
        )
        s.df_cash_flow.index.name = 'year'
        
        # Calculate IRR and NPV for w/ and w/o PCE.
        s.summary['irr'] = np.irr(s.df_cash_flow.cash_flow)
        s.summary['irr_no_pce'] = np.irr(s.df_cash_flow.cash_flow_no_pce)
        s.summary['npv'] = np.npv(s.discount_rate, s.df_cash_flow.cash_flow)
        s.summary['npv_no_pce'] = np.npv(s.discount_rate, s.df_cash_flow.cash_flow_no_pce)
        
        # Add some summary fuel and electric usage  and unit cost info
        s.summary['fuel_use_base'] = ann_base.secondary_fuel_units
        s.summary['fuel_use_hp'] =  ann_hp.secondary_fuel_units
        s.summary['fuel_use_chg'] = ann_chg.secondary_fuel_units
        s.summary['elec_use_base'] = ann_base.elec_kwh
        s.summary['elec_use_hp'] =  ann_hp.elec_kwh
        s.summary['elec_use_chg'] = ann_chg.elec_kwh
        s.summary['elec_rate_avg_base'] = ann_base.elec_dol / ann_base.elec_kwh
        s.summary['elec_rate_avg_hp'] = ann_hp.elec_dol / ann_hp.elec_kwh

In [78]:
inputs1 = dict(
    city_id=1,
    utility_id=1,
    co2_lbs_per_kwh=1.1,
    exist_heat_fuel_id=2,
    exist_unit_fuel_cost=0.97852,
    exist_fuel_use=1600,
    exist_heat_effic=.8,
    exist_kwh_per_mmbtu=8,
    exist_is_point_source=False,
    includes_dhw=True,
    dhw_occupant_count=3,
    includes_dryer=True,
    elec_use_jan=550,
    elec_use_may=400,
    hp_model_id=575,
    low_temp_cutoff=5,
    garage_stall_count=2,
    garage_heated_by_hp=False,
    bldg_floor_area=3600,
    indoor_heat_setpoint=70,
    insul_level=3,  
    pct_exposed_to_hp=0.46,
    doors_open_to_adjacent=False,
    low_temp_acceptance=2,
    capital_cost=4500,
    rebate_dol=500,
    pct_financed=0.5,
    loan_term=10,
    loan_interest=0.05,
    hp_life=14,
    op_cost_chg=10,
    sales_tax=0.02,
    discount_rate=0.05,
    inflation_rate=0.02,
    fuel_esc_rate=0.03,
    elec_esc_rate=0.02,
)

In [185]:
inputs2 = dict(
    city_id=45,
    utility_id=202,
    co2_lbs_per_kwh=1.6,
    exist_heat_fuel_id=4,
    exist_unit_fuel_cost=8.0,
    exist_fuel_use=450,
    exist_heat_effic=.86,
    exist_kwh_per_mmbtu=8,
    exist_is_point_source=True,
    includes_dhw=False,
    dhw_occupant_count=3,
    includes_dryer=False,
    elec_use_jan=550,
    elec_use_may=400,
    hp_model_id=575,
    low_temp_cutoff=5,
    garage_stall_count=0,
    garage_heated_by_hp=False,
    bldg_floor_area=800,
    indoor_heat_setpoint=70,
    insul_level=2,  
    pct_exposed_to_hp=1.0,
    doors_open_to_adjacent=False,
    low_temp_acceptance=3,
    capital_cost=6500,
    rebate_dol=0,
    pct_financed=0.0,
    loan_term=10,
    loan_interest=0.05,
    hp_life=14,
    op_cost_chg=0,
    sales_tax=0.00,
    discount_rate=0.05,
    inflation_rate=0.02,
    fuel_esc_rate=0.03,
    elec_esc_rate=0.02,
)

In [186]:
mc = HP_model(**inputs2)
mc.run()
pprint(mc.summary)
mc.df_mo_dol_base

{'co2_driving_miles_saved': 987.1566051726571,
 'co2_lbs_saved': 880.4143742987326,
 'cop': 2.836600461342373,
 'design_heat_load': 21478.25351576897,
 'design_heat_temp': -49.900000000000006,
 'elec_rate_avg_base': 0.23200883941833067,
 'elec_rate_avg_hp': 0.28991140424534884,
 'elec_use_base': 5261.587156420448,
 'elec_use_chg': 2394.8579858264475,
 'elec_use_hp': 7656.445142246896,
 'fuel': desc                                                        #1 Oil
unit                                                       gallons
btus                                                        137452
co2                                                          161.3
price_col                                                Oil1Price
dhw_effic                                                      0.6
effic_choices    [(Low 73%, 73), (Standard 80%, 80), (High Effi...
Name: 4, dtype: object,
 'fuel_use_base': 449.99999999999994,
 'fuel_use_chg': -212.53824980065627,
 'fuel_use_hp': 237.46175019934367

,elec_kwh,elec_kw,elec_dol,elec_dol_no_pce,secondary_fuel_units,secondary_fuel_dol,total_dol,total_dol_no_pce
Jan,550.000000,1.883562,138.930000,287.480004,78.222091,625.776727,764.706727,913.256731
Feb,484.208794,1.658249,109.831203,253.689640,56.307711,450.461685,560.292888,704.151325
Mar,469.285322,1.607142,106.600272,246.024945,47.888818,383.110542,489.710814,629.135487
Apr,430.885219,1.475634,98.286650,226.302652,40.416905,323.335239,421.621889,549.637891
May,400.000000,1.369863,91.600000,210.440003,21.523920,172.191357,263.791357,382.631360
Jun,352.407154,1.206874,81.296148,185.996316,7.619120,60.952959,142.249107,246.949275
Jul,358.463021,1.227613,82.607244,189.106610,8.924464,71.395713,154.002956,260.502323
Aug,370.683244,1.269463,85.252922,195.382917,13.548163,108.385305,193.638227,303.768221
Sep,396.040992,1.356305,90.742874,208.406656,25.300050,202.400397,293.143271,410.807053
Oct,437.025287,1.496662,99.615974,229.456190,32.861796,262.894368,362.510342,492.350558


In [158]:
mc.df_mo_dol_hp

,elec_kwh,elec_kw,elec_dol,elec_dol_no_pce,secondary_fuel_units,secondary_fuel_dol,total_dol,total_dol_no_pce
Jan,635.560945,3.045697,182.874102,331.424105,70.822827,566.582614,749.456716,898.006720
Feb,668.064393,2.826424,199.567873,348.117877,41.181945,329.455557,529.023430,677.573434
Mar,816.149266,2.874737,264.009337,412.559341,19.646084,157.168669,421.178006,569.728010
Apr,725.603633,2.749623,226.559663,375.109667,14.576854,116.614832,343.174495,491.724499
May,606.314911,2.574699,167.853339,316.403342,1.695108,13.560862,181.414201,329.964205
Jun,426.070938,1.691062,97.244358,223.830037,0.000000,0.000000,97.244358,223.830037
Jul,444.459354,1.719351,101.225450,233.274327,0.000000,0.000000,101.225450,233.274327
Aug,505.632318,2.123970,116.142758,264.692762,0.000000,0.000000,116.142758,264.692762
Sep,661.324561,2.301645,196.106295,344.656299,0.000000,0.000000,196.106295,344.656299
Oct,811.166379,2.738900,261.948415,410.498419,0.000000,0.000000,261.948415,410.498419


In [159]:
mc.df_cash_flow

,initial_cost,loan_cost,operating_cost,fuel_cost,elec_cost,elec_cost_no_pce,cash_flow,cash_flow_no_pce
year,,,,,,,,
0,-6500.0,0.0,0.0,0.000000,-0.000000,-0.000000,-6500.000000,-6500.000000
1,0.0,-0.0,0.0,1700.305998,-998.956033,-1201.474158,701.349965,498.831840
2,0.0,-0.0,0.0,1751.315178,-1018.935154,-1225.503641,732.380025,525.811537
3,0.0,-0.0,0.0,1803.854634,-1039.313857,-1250.013714,764.540777,553.840920
4,0.0,-0.0,0.0,1857.970273,-1060.100134,-1275.013988,797.870139,582.956285
5,0.0,-0.0,0.0,1913.709381,-1081.302137,-1300.514268,832.407244,613.195113
6,0.0,-0.0,0.0,1971.120662,-1102.928179,-1326.524553,868.192483,644.596109
7,0.0,-0.0,0.0,2030.254282,-1124.986743,-1353.055044,905.267539,677.199238
8,0.0,-0.0,0.0,2091.161911,-1147.486478,-1380.116145,943.675433,711.045765


In [88]:
mc.df_mo_en_base

,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,0.0,9.246533,0.0,10.751783,73.972266,0.0,78.222091,NaN,73.972266,1852.618197
Feb,0.0,6.656062,0.0,7.739607,53.248499,0.0,56.307711,NaN,53.248499,1333.596279
Mar,0.0,5.660876,0.0,6.582414,45.287007,0.0,47.888818,NaN,45.287007,1134.202554
Apr,0.0,4.777631,0.0,5.555384,38.221045,0.0,40.416905,NaN,38.221045,957.237177
May,0.0,2.544315,0.0,2.958506,20.354520,0.0,21.523920,NaN,20.354520,509.774217
Jun,0.0,0.900646,0.0,1.047263,7.205171,0.0,7.619120,NaN,7.205171,180.451838
Jul,0.0,1.054949,0.0,1.226685,8.439596,0.0,8.924464,NaN,8.439596,211.367714
Aug,0.0,1.601511,0.0,1.862222,12.812088,0.0,13.548163,NaN,12.812088,320.875768
Sep,0.0,2.990686,0.0,3.477542,23.925492,0.0,25.300050,NaN,23.925492,599.208378
Oct,0.0,3.884551,0.0,4.516920,31.076407,0.0,32.861796,NaN,31.076407,778.301379


In [89]:
mc.df_mo_en_hp

,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,0.874658,8.371876,92.558205,9.734739,66.975006,1.162136,70.822827,2.769581,159.533211,1825.466568
Feb,1.787997,4.868065,198.159578,5.660541,38.944520,1.168175,41.181945,2.644495,237.104098,1292.411766
Mar,3.338537,2.322338,373.572243,2.700393,18.578707,1.267596,19.646084,2.619224,392.150951,1063.014991
Apr,3.054519,1.723111,319.154568,2.003618,13.784890,1.273989,14.576854,2.805000,332.939458,855.886674
May,2.343938,0.200377,225.066419,0.232996,1.603012,1.204836,1.695108,3.052295,226.669431,400.253337
Jun,0.900646,0.000000,80.868956,0.000000,0.000000,0.484188,0.000000,3.264100,80.868956,129.390329
Jul,1.054949,0.000000,94.435928,0.000000,0.000000,0.491738,0.000000,3.274051,94.435928,151.097485
Aug,1.601511,0.000000,147.761162,0.000000,0.000000,0.854507,0.000000,3.176586,147.761162,236.417859
Sep,2.990686,0.000000,289.209061,0.000000,0.000000,0.945340,0.000000,3.030749,289.209061,462.734497
Oct,3.884551,0.000000,405.217499,0.000000,0.000000,1.242238,0.000000,2.809594,405.217499,648.347999


In [90]:
mc.city

Name                                                            Ambler
Latitude                                                        67.087
Longitude                                                     -157.859
ERHRegionID                                                          5
WAPRegionID                                                          5
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                          NaN
Oil2Price                                                          NaN
PropanePrice                                                       NaN
BirchPrice                                                         NaN
SprucePrice                                                        NaN
CoalPrice                                                          NaN
SteamPrice                                                         NaN
HotWat

In [21]:
# Model an average Enstar Home, probably somewhere between an
# insulation level 1 and 2.  Space Heating for Average Enstar
# House is about 1326 CCF, from "accessible_UA.ipynb"
m = HomeEnergyModel(**inputs)
m.insul_level = 2
m.exist_heat_effic = 0.76
m.bldg_floor_area = 2100
m.garage_stall_count = 1
m.no_heat_pump_use = True
m.calculate()
f_level2, _ = m.secondary_fuel_use()
print(m.report())

m.insul_level = 1
m.calculate()
f_level1, _ = m.secondary_fuel_use()
print(m.report())

# Assuming 2/3 Level 2 and 1/3 Level 1
print(0.67 * f_level2 + 0.33 * f_level1)

Heat Pump: 0 kWh, Secondary Fuel Use: 1,119.83 ccf
Seasonal COP: nan
Heat Pump: 0 kWh, Secondary Fuel Use: 1,508.89 ccf
Seasonal COP: nan
1248.221167662789


In [22]:
m.no_heat_pump_use = False
m.exist_heat_fuel_id = 2
m.calculate()
print(m.ser_annual)
m.df_monthly

hp_load_mmbtu              40.836879
secondary_load_mmbtu       78.081640
hp_kwh                   4218.306790
secondary_fuel_mmbtu      102.739000
secondary_kwh             624.653123
hp_kw                      13.215307
secondary_fuel_units      990.732887
total_kwh                4842.959913
co2_lbs                 17347.718953
cop                         2.837301
dtype: float64


,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,3.672106,14.695524,388.306027,19.336216,117.564193,1.674816,186.463028,2.771610,505.870221,2818.794518
Feb,4.732318,11.750202,520.081329,15.460793,94.001619,1.674816,149.091539,2.666820,614.082948,2484.403973
Mar,5.873291,8.979712,619.617631,11.815410,71.837695,1.633998,113.938383,2.778106,691.455326,2143.003865
Apr,3.965370,5.412410,377.619781,7.121592,43.299282,0.871495,68.674952,3.077655,420.919064,1296.237289
May,2.167760,2.723327,192.829052,3.583325,21.786615,0.618300,34.554723,3.294806,214.615667,655.326233
Jun,0.999401,1.192362,87.266294,1.568898,9.538897,0.489936,15.129195,3.356481,96.805191,290.046723
Jul,0.685830,0.807605,59.698311,1.062639,6.460843,0.369970,10.247239,3.367018,66.159154,197.103796
Aug,0.975151,1.161668,85.159305,1.528510,9.293340,0.481098,14.739729,3.356066,94.452645,282.733569
Sep,1.765650,2.212536,157.729926,2.911232,17.700292,0.725641,28.073599,3.280814,175.430217,533.587403
Oct,4.750804,6.588195,478.813518,8.668677,52.705558,1.500210,83.593802,2.907982,531.519076,1598.906219


In [41]:
%%timeit
m.no_heat_pump_use = False
m.calculate()

283 ms ± 6.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
m

city_id: 1
hp_model_id: 575
exist_heat_fuel_id: 2
exist_heat_effic: 0.76
exist_kwh_per_mmbtu: 8
exist_is_point_source: False
co2_lbs_per_kwh: 1.1
low_temp_cutoff: 5
garage_stall_count: 1
garage_heated_by_hp: False
bldg_floor_area: 2100
indoor_heat_setpoint: 70
insul_level: 1
pct_exposed_to_hp: 0.46
doors_open_to_adjacent: False
low_temp_acceptance: 2
hp_only: False
no_heat_pump_use: False
ua_true_up: 1.0

city:
Name                                                         Anchorage
Latitude                                                        61.152
Longitude                                                     -149.864
ERHRegionID                                                          2
WAPRegionID                                                          2
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                         3.07
Oil2Price        

In [25]:
m.df_hourly.head()

,db_temp,day_of_year,month,running,cop,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh
timestamp,,,,,,,,,,
2018-01-01 00:30:00,21.02,1,1,True,2.674526,0.009352,0.013378,1.024861,0.017603,0.107028
2018-01-01 01:30:00,19.94,1,1,True,2.629053,0.009599,0.013754,1.070064,0.018098,0.110034
2018-01-01 02:30:00,21.02,1,1,True,2.674526,0.009352,0.013378,1.024861,0.017603,0.107028
2018-01-01 03:30:00,19.04,1,1,True,2.591158,0.009804,0.014067,1.108946,0.018510,0.112539
2018-01-01 04:30:00,19.04,1,1,True,2.591158,0.009804,0.014067,1.108946,0.018510,0.112539


In [26]:
df_mo = m.df_hourly.groupby('month')[['hp_kwh', 'secondary_fuel']].sum()
df_mo.hp_kwh.plot(kind='bar')

KeyError: "Columns not found: 'secondary_fuel'"

In [ ]:
# Check whether COP is getting modeled correctly
m.df_hourly.plot(x='db_temp', y='cop', marker='.', linewidth=0)